In [1]:
import tensorflow as tf  
import numpy as np       
import pandas as pd       
from matplotlib import pyplot as plt   
import seaborn as sns  
%matplotlib inline    
import time          
from tensorflow.keras.callbacks import History       
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard       
from tensorflow.keras import metrics          
from sklearn.metrics import confusion_matrix          
from keras.callbacks import TensorBoard
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large
import pickle

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# Used to set the default drawing style of the Seaborn library
sns.set()

In [4]:
# Used to get the version number of the TensorFlow library
tf.__version__

'2.15.0'

In [5]:
# Import the necessary libraries
import platform
import tensorflow as tf

# Print Python version
print("Python version: ", platform.python_version())

# Print the TensorFlow version
print("TensorFlow version: ", tf.__version__)

# Print the current working directory
import os
print("Current working directory: ", os.getcwd())


Python version:  3.11.7
TensorFlow version:  2.15.0
Current working directory:  /root/.jupyter/张彤/模型


In [6]:
# View Linux system information This command is used in Linux to obtain the system kernel and other information. Executing this command returns detailed information about the operating system, including the kernel version, system architecture, and so on.
!uname -a

Linux w3q2ulc9.vm 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [7]:
# This command is used to view the status of the NVIDIA GPU in the current system. Executing this command will display various information about the GPU, such as model, driver version, temperature, memory usage, and so on. If an NVIDIA GPU is present on the system and the appropriate drivers and packages have been installed, this command will usually return the details of the GPU.
!nvidia-smi
# Provides some basic information about the NVIDIA GPU in the system, including the GPU name, driver version, CUDA version, and the current GPU status.

Fri Oct 25 19:19:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:09:00.0 Off |                  Off |
|  0%   23C    P8              11W / 450W |      3MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
EPOCHS = 50        # Training rounds
IMAGE_SIZE = (224,224)    # Input image size
IMAGE_PATH = "../data"
LEARNING_RATE = 1e-4          # Learning rate
BATCH_SIZE = 64            # Each time the model is trained, 64 samples are used for calculation.

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,   # The data set is divided into a training set and a validation set, with 80% used for training and 20% for validation.
    subset="training",    
    seed=123,                # Random seeds are set to ensure that the results of each partition of the data set are deterministic, so as to ensure the repeatability of the experimental results.
    image_size=IMAGE_SIZE,     # Set the image size to (224, 224)
    batch_size=BATCH_SIZE      # Set the batch size to 64
)

Found 14080 files belonging to 100 classes.
Using 11264 files for training.


2024-10-25 19:19:11.274299: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-25 19:19:11.274752: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-25 19:19:11.391794: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
vaild_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 2816 files for validation.


In [11]:
class_names = train_ds.class_names
class_names

['1',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99']

In [12]:
# Define the mean and standard deviation
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Convert the mean and standard deviation to TensorFlow tensors
mean_tensor = tf.constant(mean, dtype=tf.float32)
std_tensor = tf.constant(std, dtype=tf.float32)

# Define a function to perform a standardized operation
def normalize_image(image):
    return (image - mean_tensor) / std_tensor


In [13]:
# Image enhancement definition
train_image_augment = tf.keras.Sequential(
    [
        tf.keras.layers.Rescaling(1 / 255.0), # normalization
        tf.keras.layers.RandomRotation(factor=0.2), # Random rotation
        tf.keras.layers.RandomFlip(), # Random flip
    ]
)

valid_image_augment = tf.keras.Sequential(
    [
        tf.keras.layers.Rescaling(1 / 255.0), # normalization
    ]
)

In [14]:
# Input processing method
def process_train_input(images, labels):
    return train_image_augment(images), labels

def process_valid_input(images, labels):
    return valid_image_augment(images), labels

In [15]:
# one-hot encoding and type conversion
def convert_types_and_encode(x, y):
    y = tf.cast(y, tf.int32)  # Convert the label type to an integer
    y_one_hot = tf.one_hot(y, 100)  # Apply one-hot encoding
    return x, y_one_hot

In [16]:
# Apply data set preprocessing
train_ds = train_ds.map(convert_types_and_encode) # Convert data types and encodings
train_ds = train_ds.map(process_train_input, num_parallel_calls=tf.data.AUTOTUNE) # Application enhancement
train_ds = train_ds.prefetch(tf.data.AUTOTUNE) # Optimized loading

vaild_ds = vaild_ds.map(convert_types_and_encode) # Convert data types and encodings
vaild_ds = vaild_ds.map(process_valid_input, num_parallel_calls=tf.data.AUTOTUNE) # Application enhancement
vaild_ds = vaild_ds.prefetch(tf.data.AUTOTUNE) # Optimized loading

In [17]:
base_model = tf.keras.applications.MobileNetV3Large(include_top=False,weights='imagenet',input_shape=(*IMAGE_SIZE,3))

In [18]:
for layer in base_model.layers:
  print(layer.name)

input_1
rescaling_2
Conv
Conv/BatchNorm
tf.__operators__.add
re_lu
tf.math.multiply
multiply
expanded_conv/depthwise
expanded_conv/depthwise/BatchNorm
re_lu_1
expanded_conv/project
expanded_conv/project/BatchNorm
expanded_conv/Add
expanded_conv_1/expand
expanded_conv_1/expand/BatchNorm
re_lu_2
expanded_conv_1/depthwise/pad
expanded_conv_1/depthwise
expanded_conv_1/depthwise/BatchNorm
re_lu_3
expanded_conv_1/project
expanded_conv_1/project/BatchNorm
expanded_conv_2/expand
expanded_conv_2/expand/BatchNorm
re_lu_4
expanded_conv_2/depthwise
expanded_conv_2/depthwise/BatchNorm
re_lu_5
expanded_conv_2/project
expanded_conv_2/project/BatchNorm
expanded_conv_2/Add
expanded_conv_3/expand
expanded_conv_3/expand/BatchNorm
re_lu_6
expanded_conv_3/depthwise/pad
expanded_conv_3/depthwise
expanded_conv_3/depthwise/BatchNorm
re_lu_7
expanded_conv_3/squeeze_excite/AvgPool
expanded_conv_3/squeeze_excite/Conv
expanded_conv_3/squeeze_excite/Relu
expanded_conv_3/squeeze_excite/Conv_1
tf.__operators__.add_1

In [19]:
trainable = False
for layer in base_model.layers:
    if layer.name == 'block_14_expand':
        trainable = True
    layer.trainable = trainable

In [20]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='softmax'),
])

In [21]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 47040)             0         
                                                                 
 dense (Dense)               (None, 1024)              48169984  
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 100)               102500    
                                                                 
Total params: 51272932 (195.59 MB)
Trainable params: 4

In [23]:
# Definition optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Defined loss function
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Define evaluation indicators
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
]

# Compilation model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

In [24]:
log_dir = "../Running result/MobileNetv3/MobileNetv3"

In [25]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [26]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=1e-8)

In [27]:
history = model.fit(train_ds,epochs=EPOCHS,validation_data=vaild_ds,callbacks=[lr_scheduler,tensorboard_callback])

Epoch 1/50


2024-10-25 19:19:30.850228: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-25 19:19:30.987331: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-25 19:19:31.977855: I external/local_xla/xla/service/service.cc:168] XLA service 0x86fb890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-25 19:19:31.977942: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-25 19:19:31.977958: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-25 19:19:32.000131: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729855172.200288    1638 device_compiler.

176/176 [==============================] - 19s 66ms/step - loss: 4.3347 - categorical_accuracy: 0.0695 - precision: 0.1284 - recall: 0.0017 - val_loss: 5.5122 - val_categorical_accuracy: 0.0146 - val_precision: 0.1364 - val_recall: 0.0053 - lr: 1.0000e-04
Epoch 2/50
176/176 [==============================] - 10s 52ms/step - loss: 3.9937 - categorical_accuracy: 0.1064 - precision: 0.3548 - recall: 0.0049 - val_loss: 5.4877 - val_categorical_accuracy: 0.0408 - val_precision: 0.1000 - val_recall: 0.0032 - lr: 1.0000e-04
Epoch 3/50
176/176 [==============================] - 10s 51ms/step - loss: 3.8534 - categorical_accuracy: 0.1254 - precision: 0.4775 - recall: 0.0075 - val_loss: 7.7015 - val_categorical_accuracy: 0.0266 - val_precision: 0.0269 - val_recall: 0.0174 - lr: 1.0000e-04
Epoch 4/50
176/176 [==============================] - 10s 51ms/step - loss: 3.7607 - categorical_accuracy: 0.1418 - precision: 0.5200 - recall: 0.0127 - val_loss: 6.1470 - val_categorical_accuracy: 0.0455 - val

In [28]:
import pandas as pd  # Import the Pandas library
import os

# Define save path
model_dir = "../Running result/Running result/MobileNetV3"
excel_file_path = os.path.join(model_dir, "MobileNetv3.xlsx")  # Path to save the Excel file
model_file_path = os.path.join(model_dir, "MobileNetv3.h5")  # Path to save the model

# Check if the save directory exists, create it if not
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the training history to an Excel file
history_df = pd.DataFrame(history.history)  
history_df.to_excel(excel_file_path, index=False)
print(f"Training history saved to: {excel_file_path}")

# Save the model to the specified path
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Training history saved to: ../Running result/Running result/MobileNetV3/MobileNetv3.xlsx


/root/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to: ../Running result/Running result/MobileNetV3/MobileNetv3.h5


In [29]:
import os
import time
import numpy as np
import tensorflow as tf
import random
from PIL import Image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input

# Force GPU operations to be synchronized to ensure accurate timing
tf.config.experimental.set_synchronous_execution(True)

# Set TensorFlow log level to output only errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR

# Create Model (MobileNetV3Large)
def create_model():
    inputs = Input(shape=(224, 224, 3))
    # Use MobileNetV3Large as the base model without the top layers
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights='imagenet', input_tensor=inputs)
    
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)  # Global average pooling
    outputs = Dense(100, activation='softmax')(x)  # Assume 100-class classification
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Get Image Files 
def get_image_files(dataset_dir):
    image_files = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    return image_files

# Process Image 
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# GPU Synchronization and Cache Clearing 
def gpu_synchronize_and_clear():
    tf.config.experimental.set_synchronous_execution(True)  # Force GPU synchronization
    tf.keras.backend.clear_session()  # Clear GPU cache

# Preload All Images to Memory 
def preload_all_images_to_memory(image_files):
    """Preload all images into memory to avoid frequent disk access during inference."""
    preloaded_images = {}
    for image_file in image_files:
        img_array = load_and_preprocess_image(image_file)
        preloaded_images[image_file] = img_array
    return preloaded_images

# Inference Function 
def inference_with_separate_timing(model, images, num_images):
    gpu_synchronize_and_clear()  # Clear cache and synchronize GPU

    # Perform inference and time it
    t1 = time.time()
    model.predict(images)
    gpu_synchronize_and_clear()  # Synchronize GPU again

    t2 = time.time()
    inference_time = t2 - t1  # Record inference time
    return inference_time

# Single Image Inference 
def single_image_inference(model, image_files):
    random_image_file = random.choice(image_files)
    
    # Record image loading and preprocessing time
    image_data = load_and_preprocess_image(random_image_file)
    
    # Perform inference and record inference time
    inference_time = inference_with_separate_timing(model, image_data, 1)
    return inference_time, random_image_file

# Batch Image Inference 
def batch_image_inference_and_async_loading(model, preloaded_images, image_files, batch_size):
    t1 = time.time()
    for img in random.sample(image_files, batch_size):
        img = preloaded_images[img]
        model.predict(img)
    t2 = time.time()
    inference_time = t2 - t1
    average_time_per_image = inference_time / batch_size
    return inference_time, average_time_per_image

# Main Program Entry 
if __name__ == "__main__":
    dataset_dir = "../data"  
    model = create_model()

    # Model warm-up
    model(np.zeros((1, 224, 224, 3)))

    # Get image files
    image_files = get_image_files(dataset_dir)

    # Preload all images into memory
    preloaded_images = preload_all_images_to_memory(image_files)

    # Calculate inference time for 1 image, 500 images, and 1000 images
    for batch_size in [1, 500, 1000]:
        if batch_size == 1:
            inference_time, image_used = single_image_inference(model, image_files)
            print(f"Single image inference time: {inference_time * 1000:.4f} ms")
        else:
            inference_time, avg_time_per_image = batch_image_inference_and_async_loading(model, preloaded_images, image_files, batch_size)
            print(f"Batch of {batch_size} images inference time: {inference_time * 1000:.4f} ms")
            print(f"Average time per image in batch of {batch_size}: {avg_time_per_image * 1000:.4f} ms")


1/1 [==============================] - 1s 1s/step
Single image inference time: 1138.1013 ms
1/1 [==============================] - 0s 43ms/step
Batch of 500 images inference time: 52078.8488 ms
Average time per image in batch of 500: 104.1577 ms
1/1 [==============================] - 0s 37ms/step
Batch of 1000 images inference time: 107797.1742 ms
Average time per image in batch of 1000: 107.7972 ms
